In [79]:
# import necessary libraries
import pandas as pd
import re
from airportsdata import load
from actions.done.CreateZFWMessageAction import CreateZFWMessageAction
from actions.done.UpdateEstimatesAction import UpdateEstimatesAction
from actions.done.CalculateWeightAndTrimAction import CalculateWeightAndTrimAction
import json

def load_config(path):
    with open(path, 'r') as file:
        return json.load(file)

config_path = '../pandas_config.json'
config = load_config(config_path)
parquet_config = config['parquet']

# Setting up the engine as a global constant
ENGINE = parquet_config['engine']
COMPRESSION = parquet_config['compression']
FILE_PATH = "../data"

In [80]:
df = pd.read_parquet(f"{FILE_PATH}/data_parquet/processed_data_combined.parquet", engine=ENGINE)

In [81]:
df = df[df["action_name"].isin(["CalculateWeightAndTrimAction"])]

In [51]:
df = df[df["action_name"].isin(["CreateZFWMessageAction", "UpdateEstimatesAction"])]

In [82]:
flight_numbers = df['flight_number'].unique()
flight_numbers = [flight_number for flight_number in flight_numbers if flight_number != None]

In [85]:
testdf = df[df['flight_number']==1116]

In [86]:
testdf

,id,creation_time,formatted_creation_time,airline_code,flight_number,flight_date,departure_airport,departure_city,departure_country,departure_lat,departure_lon,user_name,action_name,stepID,action_mode,log_level,entry_details,arrival_airport
52190,137526011,2024-04-30 04:05:51,30-04-2024 04:05:51,MN,1116,30,IAD,Washington,US,38.9,-77.5,service-acco,CalculateWeightAndTrimAction,dd9471a8de659336,Received,INFO,com.systemone.lc2.common.dto.SingleAttributeDT...,DUB
52191,137526012,2024-04-30 04:05:51,30-04-2024 04:05:51,MN,1116,30,IAD,Washington,US,38.9,-77.5,service-acco,CalculateWeightAndTrimAction,dd9471a8de659336,Received,INFO,com.systemone.lc2.common.dto.SingleAttributeDT...,DUB
52192,137526013,2024-04-30 04:05:51,30-04-2024 04:05:51,MN,1116,30,IAD,Washington,US,38.9,-77.5,service-acco,CalculateWeightAndTrimAction,dd9471a8de659336,Saved,INFO,START_WI weight : 52301.00 KG ...,DUB
52193,137526014,2024-04-30 04:05:51,30-04-2024 04:05:51,MN,1116,30,IAD,Washington,US,38.9,-77.5,service-acco,CalculateWeightAndTrimAction,dd9471a8de659336,Saved,INFO,START_WI weight : 52301.00 KG ...,DUB
52218,137528943,2024-04-30 04:35:48,30-04-2024 04:35:48,MN,1116,30,IAD,Washington,US,38.9,-77.5,service-acco,CalculateWeightAndTrimAction,77ccc18aeef4b76b,Received,INFO,com.systemone.lc2.common.dto.SingleAttributeDT...,DUB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79133,138647226,2024-05-07 09:08:29,07-05-2024 09:08:29,MN,1116,7,IAD,Washington,US,38.9,-77.5,service-acco,CalculateWeightAndTrimAction,54fd59d9583dffe8,Saved,INFO,START_WI weight : 52174.00 KG ...,DUB
79142,138647414,2024-05-07 09:10:36,07-05-2024 09:10:36,MN,1116,7,IAD,Washington,US,38.9,-77.5,service-acco,CalculateWeightAndTrimAction,6516672cf7d95c63,Received,INFO,com.systemone.lc2.common.dto.SingleAttributeDT...,DUB
79143,138647415,2024-05-07 09:10:36,07-05-2024 09:10:36,MN,1116,7,IAD,Washington,US,38.9,-77.5,service-acco,CalculateWeightAndTrimAction,6516672cf7d95c63,Received,INFO,com.systemone.lc2.common.dto.SingleAttributeDT...,DUB
79144,138647416,2024-05-07 09:10:36,07-05-2024 09:10:36,MN,1116,7,IAD,Washington,US,38.9,-77.5,service-acco,CalculateWeightAndTrimAction,6516672cf7d95c63,Saved,INFO,START_WI weight : 52174.00 KG ...,DUB


In [83]:
estimated_zfws = {}
actual_zfws = {}
for flight in flight_numbers[:100]:
    subset_df = df[df['flight_number'] == flight]
    subset_df.sort_index(inplace=True)
    actions = subset_df['action_name'].values
    entriy_details = subset_df["entry_details"].values
    temp_estimated_zfws = []
    temp_actual_zfws = []
    for entry, action in zip(entriy_details, actions):
        if action == "CreateZFWMessageAction":
            estimated_zfw, actual_zfw = CreateZFWMessageAction(entry)
            if estimated_zfw != None:
                temp_estimated_zfws.append(estimated_zfw)
            if actual_zfw != None:
                temp_actual_zfws.append(actual_zfw)
        if action == "UpdateEstimatesAction":
            estimated_zfw = UpdateEstimatesAction(entry)
            if estimated_zfw != None:
                temp_estimated_zfws.append(estimated_zfw)
            if actual_zfw != None:
                temp_actual_zfws.append(actual_zfw)

        if action == "CalculateWeightAndTrimAction":
            actual_zfw = CalculateWeightAndTrimAction(entry)
            if actual_zfw != None:
                temp_actual_zfws.append(actual_zfw)
    estimated_zfws[flight] = temp_estimated_zfws
    actual_zfws[flight] = temp_actual_zfws
    #estimated_zfws.append(temp_estimated_zfws)
    #actual_zfws.append(temp_actual_zfws)

In [84]:
actual_zfws

{1581: [],
 1116: [65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65324,
  65384,
  65697,
  67386,
  65384,
  66310,
  66773,
  67999,
  67999,
  67999,
  68134,
  68124,
  68124,
  68124,
  68124,
  68124,
  68124,
  68124,
  68124,
  68124,
  53789,
  68124,
  68124,
  68124,
  68124,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  68437,
  

In [69]:
estimated_zfws_tuples = [(k, v) for k, v in estimated_zfws.items()]
actual_zfws_tuples = [(k, v) for k, v in actual_zfws.items()]

# Create DataFrame
estimated_zfws_df = pd.DataFrame(estimated_zfws_tuples, columns=['flight_number', 'estimated_zfws'])
actual_zfws_df = pd.DataFrame(actual_zfws_tuples, columns=['flight_number', 'actual_zfws'])
weights = pd.merge(estimated_zfws_df, actual_zfws_df, on='flight_number', how="outer")


In [73]:
weights.to_parquet("../data/data_parquet/weights.parquet", engine=ENGINE, compression=COMPRESSION)